In [1]:
import pandas as pd
import sqlite3

In [2]:
sp_dir = '/Users/rwang/RMI/Climate Action Engine - Documents/OCI Phase 2'
connection = sqlite3.connect(sp_dir+"/OCI_Database.db")

In [3]:
field_name = pd.read_sql('select Field_name, "Field location (Country)" from upstream_results',connection)

In [5]:
field_name = field_name.drop_duplicates()

In [7]:
field_name.shape

(670, 2)

In [8]:
field_name.to_csv('/Users/rwang/RMI/Climate Action Engine - Documents/ClimateTRACE Phase 4/oldoci.csv')

In [9]:
OCI_info = pd.read_csv('/Users/rwang/RMI/Climate Action Engine - Documents/OCI Phase 2/Upstream/upstream_data_pipeline_sp/Postprocessed_outputs_2/infobase_withcommas_2022-06-06.csv')

In [11]:
info_base_cols = ['Country', 'Field Name', 'Assay Name', '2020 Total Oil and Gas Production Volume (boe)', 'Location', 'Max Depth(ft)', 'Gas shipped as LNG', 'Enhanced recovery', 'Fracked', 'Default Refinery Configuration',
'Heating Value Processed Oil and Gas (MJ/d)', 'Years in Production', 'Number of Producing Wells', '2020 Crude Production Volume (bbl)', 
'Region', 'Latitude', 'Longitude', 'API Gravity', 'Sulfur Content Weight Percent', 'Water-to-oil Ratio (bbl water/bbl oil)', 
'Gas-to-Oil Ratio (scf/bbl)', 'Flaring-to-Oil Ratio (scf flared/bbl)', 'Upstream Methane Intensity (kgCH4/boe)', 
'Midstream Methane Intensity (kgCH4/boe)', 'Downstream Methane Intensity (kgCH4/boe)', 'Total Methane Intensity (kgCH4/boe)',
'Upstream Methane Emission Rate (NGSI Standard %)', 'Upstream Methane Emission Rate (gCH4/Total MJ Produced)', 'Gas composition H2S',
'Gas composition CO2', 'Gas composition C1', 'Flare Rate', 'OPEC', 'Oil or Gas', 'Resource Type', 'Sour or Sweet', 'descriptor']
#OCI_info.drop(columns = '_merge', inplace = True)
coordinates = pd.read_csv(sp_dir + '/Upstream/upstream_data_pipeline_sp/Inputs/coordinates.csv')
match_table = pd.read_csv(sp_dir + '/Upstream/upstream_data_pipeline_sp/Inputs/field_match.csv')
select_coordinates = match_table.merge(coordinates,left_on = ['matched_field'],right_on = ['Field Name'], how = 'left',indicator = True)
select_coordinates[select_coordinates['_merge']!='both']
select_coordinates = select_coordinates[['Field Name_x','Country_y','Region','Latitude','Longitude']]
select_coordinates.rename(columns = {'Field Name_x':'Field Name'}, inplace = True)
OCI_info = OCI_info.merge(select_coordinates, how = 'left',indicator = True)
def region_mod(x):
    if x in['Asia','Oceania']:
        return 'Asia-Pacific'
    elif x in(['Africa','South America','North America']):
        return x+'n'
    elif x in(['Middle East','Caribbean','Former Soviet Union']):
        return x
    else:
        return x

OCI_info['Region_m']=OCI_info['Region'].apply(region_mod)

OCI_info['descriptor']= (
        'The '
        + OCI_info['Field Name']
        +' field is located in '
        + OCI_info['Country']
        + '. This '
        + OCI_info['Region_m']
        + ' region asset is classified as '
        + OCI_info['Resource Type'].apply(lambda x: x.lower())
        + '. All produced liquids are processed in a '
        + OCI_info['Default Refinery Configuration'].apply(lambda x: x.lower())
        + ' refinery assuming the following oil assay: '
        + OCI_info['Assay Name']
        + '. Following are the detailed resource characteristics modeled in the OCI+.')

OCI_info.drop(columns = 'Region_m',inplace = True)

OCI_info.rename(columns ={'Max Depth (ft)':'Max Depth(ft)','API gravity': 'API Gravity',
'Sulfur wt percent': 'Sulfur Content Weight Percent', 'Resource type': 'Resource Type'},inplace = True)
infobase = OCI_info[info_base_cols][OCI_info['gwp']==100]
info_100_cols=['Country', 'Field Name', 'Upstream: Exploration (kgCO2eq/boe)', 'Upstream: Drilling & Development (kgCO2eq/boe)', 
'Upstream: Crude Production & Extraction (kgCO2eq/boe)', 'Upstream: Surface Processing (kgCO2eq/boe)', 
'Upstream: Maintenance (kgCO2eq/boe)', 'Upstream: Waste Disposal (kgCO2eq/boe)', 'Upstream: Crude Transport (kgCO2eq/boe)', 
'Upstream: Other Small Sources (kgCO2eq/boe)', 'Upstream: Offsite emissions credit/debit (kgCO2eq/boe)', 
'Upstream: Carbon Dioxide Sequestration (kgCO2eq/boe)', 'Upstream Carbon Intensity (kgCO2eq/boe)', 
'Midstream: Electricity (kgCO2eq/boe)', 'Midstream: Heat (kgCO2eq/boe)', 'Midstream: Steam (kgCO2eq/boe)', 
'Midstream: Hydrogen via SMR (kgCO2eq/boe)', 'Midstream: Hydrogen via CNR (kgCO2eq/boe)', 
'Midstream: Other Emissions (kgCO2eq/boe)', 'Midstream Carbon Intensity (kgCO2eq/boe)', 
'Downstream: Transport of Petroleum Products to Consumers (kgCO2eq/boe)', 
'Downstream: Transport of LNG to Consumers (kgCO2eq/boe)', 
'Downstream: Transport of Pipeline Gas to Consumers (kgCO2eq/boe)', 
'Downstream: Gasoline for Cars (kgCO2eq/boe)', 'Downstream: Jet Fuel for Planes (kgCO2eq/boe)', 
'Downstream: Diesel for Trucks and Engines (kgCO2eq/boe)', 'Downstream: Fuel Oil for Boilers (kgCO2eq/boe)', 
'Downstream: Petroleum Coke for Power (kgCO2eq/boe)', 'Downstream: Liquid Heavy Ends for Ships (kgCO2eq/boe)', 
'Downstream: Natural Gas Liquids (kgCO2eq/boe)', 'Downstream: Liquefied Petroleum Gases (kgCO2eq/boe)', 
'Downstream: Petrochemical Feedstocks (kgCO2eq/boe)', 'Downstream: Natural Gas (kgCO2eq/boe)', 
'Downstream Carbon Intensity (kgCO2eq/boe)', 'Total Emission Carbon Intensity (kgCO2eq/boe)', 
'Industry GHG Responsibility (kgCO2eq/boe)', 'Consumer GHG Responsibility (kgCO2eq/boe)']
info100 = OCI_info[OCI_info['gwp']==100][info_100_cols]
info20 = OCI_info[OCI_info['gwp']==20][info_100_cols]
infobase['Fracked'] = infobase['Fracked'].apply(lambda x: bool(x))
infobase['Gas shipped as LNG'] = infobase['Gas shipped as LNG'].apply(lambda x: bool(x))
infobase.to_csv('/Users/rwang/Documents/oci/basedata/infobase.csv')
info20.to_csv('/Users/rwang/Documents/oci/basedata/info20.csv')
info100.to_csv('/Users/rwang/Documents/oci/basedata/info100.csv')

KeyError: 'Resource type'